In [1]:
import pandas as pd
import requests
import pprint
from pymongo import MongoClient
from bs4 import BeautifulSoup
import json
import time
from lxml import html
from io import StringIO
from datetime import datetime

In [23]:
pd.set_option('display.max_columns', None)

# Yuli Gurriel
This notebook i'll attempt to make a few lists and iterate through them for the scrape rather than do it manually for each year. 


In [11]:
urls = ['https://www.retrosheet.org/boxesetc/2016/Igurry0010012016.htm',
        'https://www.retrosheet.org/boxesetc/2017/Igurry0010022017.htm']

csv_names = ['csv_from_scrape/gurriel_2016_df.csv',
             'csv_from_scrape/gurriel_2017_df.csv']

years = ['2016', '2017']

In [34]:
for idx, url in enumerate(urls):
    
    # batting averages

    xpath = "//pre[5]/text()"
    r = requests.get(url)
    page = html.parse(StringIO(r.text))
    batter = page.xpath(xpath)
    
    batter_game_data_only = []
    for a, row in enumerate(batter):
        if a % 2 == 0:
            batter_game_data_only.append(row)
    
    batter_game_data_cleaned = []
    for row in batter_game_data_only[1:]:
        if len(row) > len(batter_game_data_only[2]):
            batter_game_data_cleaned.append(row[0:112])
        else:
            batter_game_data_cleaned.append(row)
        
    batter_averages = [float(row[87:91]) for row in batter_game_data_cleaned]

    # dates played this year

    soup = BeautifulSoup(r.text, 'html.parser')
    
    link_text = []
    for a in soup.find_all('a'):
        link_text.append(a.string)
        
    batter_dates = []
    for i in link_text:
        if years[idx] in i:
            batter_dates.append(i)
            
    batter_dates_cleaned = []
    for date in batter_dates:
        batter_dates_cleaned.append(date.replace(' ', '0'))
        
    batter_dates_final = []
    for date in batter_dates_cleaned:
        batter_dates_final.append(datetime.strftime(datetime.strptime(date, '%m-%d-%Y'), '%m-%d'))
        
    batter_df = pd.DataFrame(batter_averages, batter_dates_final).reset_index()
    batter_df.columns = [years[idx], 'Batting Average']
    batter_df.to_csv(csv_names[idx])